# Downloading comments about the video
## Analysis of Unstructured Data
### Anna Szymanek 230042

In [1]:
from googleapiclient.discovery import build
import pandas as pd

In [2]:
video_title = 'Why Pakistan Can Become the #1 Travel Destination in the World'
csv_file_name = 'eva_comment.csv'

In [3]:
youTubeApiKey = 'myApiKey'
youtube = build('youtube', 'v3', developerKey = youTubeApiKey)

In [4]:
youtube.search()

In [5]:
query_results = youtube.search().list(part = 'snippet', q = video_title, order = 'relevance', type = 'video', regionCode = 'pl' , relevanceLanguage = 'en', safeSearch = 'moderate').execute()

In [6]:
video_id = []
channel = []
video_title = []
video_desc = []
for item in query_results['items']:
    video_id.append(item['id']['videoId'])
    channel.append(item['snippet']['channelTitle'])
    video_title.append(item['snippet']['title'])
    video_desc.append(item['snippet']['description'])

I only want first video.

In [7]:
video_id = video_id[0]
channel = channel[0]
video_title = video_title[0]
video_desc = video_desc[0]

In [8]:
video_id_pop = []
channel_pop = []
video_title_pop = []
video_desc_pop = []
comments_pop = []
comment_id_pop = []
reply_count_pop = []
like_count_pop = []

comments_temp = []
comment_id_temp = []
reply_count_temp = []
like_count_temp = []

In [9]:
nextPage_token = None

while 1:
    response = youtube.commentThreads().list(part = 'snippet', videoId = video_id, maxResults = 100, order = 'relevance', textFormat = 'plainText', pageToken = nextPage_token).execute()
    nextPage_token = response.get('nextPageToken')
    for item in response['items']:
        comments_temp.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
        comment_id_temp.append(item['snippet']['topLevelComment']['id'])
        reply_count_temp.append(item['snippet']['totalReplyCount'])
        like_count_temp.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
        comments_pop.extend(comments_temp)
        comment_id_pop.extend(comment_id_temp)
        reply_count_pop.extend(reply_count_temp)
        like_count_pop.extend(like_count_temp)

        video_id_pop.extend([video_id]*len(comments_temp))
        channel_pop.extend([channel]*len(comments_temp))
        video_title_pop.extend([video_title]*len(comments_temp))
        video_desc_pop.extend([video_desc]*len(comments_temp))
        
 
    if nextPage_token is None:
        break

In [10]:
output_dict = {
    'Channel': channel_pop,
    'Video Title': video_title_pop,
    'Video Description': video_desc_pop,
    'Video ID': video_id_pop,
    'Comment': comments_pop,
    'Comment iD': comment_id_pop,
    'Replies': reply_count_pop,
    'Likes': like_count_pop
}
output_df = pd.DataFrame(output_dict, columns = output_dict.keys())

In [11]:
output_df.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment iD,Replies,Likes
0,Eva zu Beck,Why Pakistan Can Become the #1 Travel Destinat...,"This isn't my typical video. Yesterday, I had ...",AeSadjRblyM,Thank you everyone for your wonderful outpouri...,Ugwm5i5wTuGH8023nIt4AaABAg,475,5037
1,Eva zu Beck,Why Pakistan Can Become the #1 Travel Destinat...,"This isn't my typical video. Yesterday, I had ...",AeSadjRblyM,Thank you everyone for your wonderful outpouri...,Ugwm5i5wTuGH8023nIt4AaABAg,475,5037
2,Eva zu Beck,Why Pakistan Can Become the #1 Travel Destinat...,"This isn't my typical video. Yesterday, I had ...",AeSadjRblyM,We share your vision Eva. Thanks for sharing i...,Ugyk-49uVzAz23hlfLR4AaABAg,17,841
3,Eva zu Beck,Why Pakistan Can Become the #1 Travel Destinat...,"This isn't my typical video. Yesterday, I had ...",AeSadjRblyM,Thank you everyone for your wonderful outpouri...,Ugwm5i5wTuGH8023nIt4AaABAg,475,5037
4,Eva zu Beck,Why Pakistan Can Become the #1 Travel Destinat...,"This isn't my typical video. Yesterday, I had ...",AeSadjRblyM,We share your vision Eva. Thanks for sharing i...,Ugyk-49uVzAz23hlfLR4AaABAg,17,841


In [12]:
duplicates = output_df[output_df.duplicated('Comment')]
duplicates.shape[0]

1999040

In [13]:
output_df.shape[0] - duplicates.shape[0]

1960

In [14]:
unique_df = output_df.drop_duplicates(subset = ['Comment'])

In [15]:
unique_df.shape

(1960, 8)

In [16]:
unique_df.to_csv(csv_file_name, index = False)